## CTOT Monotonicity Lemma (1)

Given two subsequences $s$ and $s'$ with identical aircraft in the same order, differing only in their take-off times (e.g., due to different aircraft preceding $s$ and $s'$), if $t_x \le t'_x$ for all aircraft $x \in s, s'$,
then the delay (or CTOT) cost for each individual aircraft in $s$ will be no worse than its delay (or CTOT) cost in $s'$,
and the total delay (or CTOT) cost summed over all aircraft in $s$ will be no worse than the total delay (or CTOT) cost summed over all aircraft in $s'$. Initialise Z3 and create a `Solver` instance.

In [1]:
from z3 import *
s = Solver()


First, we define $s: \mathbb{Z} \to \mathbb{R}$ (`S1`) and $s': \mathbb{Z} \to \mathbb{R}$ (`S2`) as arrays. We also define an array to define timeslots, $L_c : \mathbb{Z} \to \mathbb{R}$ (`Lc`). Constant penalty weights ($\omega_1$ through $\omega_4$) for CTOT compliance are defined as real numbers.


In [2]:
S1  = Array('S1', IntSort(), RealSort())   # Times in s
S2 = Array('S2', IntSort(), RealSort())   # Times in s'
Lc = Array('Lc', IntSort(), RealSort())   # CTOT slot times

ω1, ω2, ω3, ω4 = Reals('ω1 ω2 ω3 ω4')

s.add(ω1 >= 0, ω2 >= 0, ω3 >= 0, ω4 >= 0)

s.add(ω1 <= ω3, ω2 <= ω4)                 # Constraints on what ω can be.

Then, define the cost function for CTOT violations, as defined in ([De Maere et al., 2017](https://pubsonline.informs.org/doi/epdf/10.1287/trsc.2016.0733)). This is defined in Z3 as follows, as `C`.

$$
C(t_i, l_i^c) =
\begin{cases}
  0, & \text{if } t_i \le l_i^c, \\[6pt]
  \omega_1 (t_i - l_i^c) + \omega_2, & \text{if } l_i^c < t_i \le l_i^c + 300, \\[6pt]
  \omega_3 (t_i - l_i^c) + \omega_4, & \text{if } t_i > l_i^c + 300.
\end{cases}
$$

In [3]:
def C(t, lc):
    return If(t <= lc,
              RealVal(0),
              If(t <= lc + 300,
                 ω1 * (t - lc) + ω2,
                 ω3 * (t - lc) + ω4))

Then we formalize the following lemma into Z3. This states that for all time aircraft $i$, if $i$ is scheduled earlier in $s$ than $s'$ then the CTOT violation cost (for that individual aircraft) is less in $s$ than $s'$. This wrapped in negation, so that Z3 finds a counter example.

$$
\neg (\forall i, s[i] \leq s'[i] \implies C(s[i], L[i]) \leq C(s'[i], L[i]))
$$



In [4]:
i = Int('i')
seq_lemma = ForAll(
    [i],
    Implies(
        Select(S1, i) <= Select(S2, i),
        C(Select(S1, i), Select(Lc, i)) <= C(Select(S2, i), Select(Lc, i))
    )
)
s.add(Not(seq_lemma))

Now, check the result, if the lemma holds this should be unsat.

In [5]:
if s.check() == z3.sat:
    model = s.model()
    print("\nLemma holds SAT (counterexample found).\nModel:")
    for d in model.decls():
        print(f" - {d.name()} = {model[d]}")
elif s.check() == z3.unsat:
    print("\nLemma holds: UNSAT (no counterexample found)")


Lemma holds: UNSAT (no counterexample found)
